In [21]:
dstype = '20NG' 
mname = 'debertaV3'

In [22]:
 
modelpath = 'microsoft/deberta-v3-base'
# modelpath = "bert-base-uncased"


datapath = None
saveDIR = f"/home/bhairavi/om/om5/{dstype}/{mname}_{dstype}"
print(saveDIR)
# %%

/home/bhairavi/om/om5/20NG/debertaV3_20NG


In [23]:


# %%
# %%
 
import os
import torch  

torch.cuda.empty_cache() 

import pandas as pd
from datasets import Dataset, DatasetDict, load_metric
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
 

 


# %%
 


In [24]:
from datasets import load_dataset

dataset = load_dataset('rungalileo/20_Newsgroups_Fixed')

In [25]:
dataset


DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label'],
        num_rows: 11314
    })
    test: Dataset({
        features: ['id', 'text', 'label'],
        num_rows: 7532
    })
})

In [26]:
import pandas as pd

In [27]:
test_df = pd.DataFrame(dataset['test'])
train_df = pd.DataFrame(dataset['train'])

In [28]:
train_df['split'] = 'train'

test_df['split'] = 'test'
 

In [29]:
df = pd.concat([train_df, test_df], ignore_index=True)

In [30]:
 
df.shape

(18846, 4)

In [31]:
df.dropna(inplace=True)
df.shape

(17683, 4)

In [32]:
df.columns

Index(['id', 'text', 'label', 'split'], dtype='object')

In [12]:
df['text'][0] , df['label'][0] 

('I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.',
 'rec.autos')

In [33]:
df['label'].nunique()

20

In [34]:

# %%

# %%
df.sample(5)


# %%


,id,text,label,split
1511,1511,...His account that is.\n\nMany important issu...,talk.politics.misc,train
7196,7196,"Well, after a lot of trawling through archives...",sci.crypt,train
2616,2616,: \n: Where could I find a description of the ...,comp.graphics,train
13378,2064,"\nWell, -I've- been reading t.p.m. for a while...",talk.politics.mideast,test
11829,515,The Microsoft Windows Device Driver Kit (DDK) ...,comp.os.ms-windows.misc,test


In [35]:
df.drop(columns=["id"], inplace=True)
 


In [36]:
df[0:3]

,text,label,split
0,I was wondering if anyone out there could enli...,rec.autos,train
1,A fair number of brave souls who upgraded thei...,comp.sys.mac.hardware,train
2,"well folks, my mac plus finally gave up the gh...",comp.sys.mac.hardware,train


In [38]:


# %%

# %%
from sklearn.preprocessing import LabelEncoder

# %%
le = LabelEncoder()
df['target'] = le.fit_transform(df['label'])

 

# %%

# %%
numlabel = df['target'].nunique()
numlabel


20

In [ ]:
df['target'].nunique(), df['label'].nunique()

In [41]:
df.columns

Index(['text', 'label', 'split', 'target'], dtype='object')

In [40]:
numlabel = df['target'].nunique()
numlabel


20

In [42]:
df['text'] = df['text'].apply(lambda x: str(x)[:512] if isinstance(x, float) else x[:512])


In [43]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

device = "cuda"  # the device to load the model onto

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(modelpath)
model = AutoModelForSequenceClassification.from_pretrained(modelpath, num_labels=numlabel)

# Move the model to the specified device
model.to(device)

/home/bhairavi/om/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/home/bhairavi/om/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint 

DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine

In [ ]:
# df['token_length'] = df['text'].apply(lambda x: len(tokenizer.tokenize(x)))

# # Calculate the maximum token length
# max_length = df['token_length'].max()

# # Calculate the next maximum token length
# next_max_token_length = df['token_length'].nlargest(2).iloc[1]

# # Calculate the average token length
# average_token_length = df['token_length'].mean()

# # Display the results
# print(f"Maximum token length: {max_length}")
# print(f"Next maximum token length: {next_max_token_length}")
# print(f"Average token length: {average_token_length:.2f}")

In [44]:
max_length = 512

In [45]:
train_df = df[df['split'] == 'train'].drop(columns=['split'])

test_df = df[df['split'] == 'test'].drop(columns=['split'])
 

In [46]:
train_df.shape, test_df.shape

((10664, 3), (7019, 3))

In [47]:
train_val_df = train_df

In [48]:

# %%
from sklearn.model_selection import StratifiedShuffleSplit

# Splitting off the test set with 5% of the data
# sss = StratifiedShuffleSplit(n_splits=1, test_size=0.05, random_state=42)  # 5% for test
# for train_val_idx, test_idx in sss.split(df, df['target']):
#     train_val_df = df.iloc[train_val_idx]
#     test_df = df.iloc[test_idx]

# Further split train_val_df into train and validation sets with validation set being 15.79% of the remaining data
# (which is equivalent to 15% of the original dataset size)
sss_val = StratifiedShuffleSplit(n_splits=1, test_size=0.15, random_state=42)  # ~15.79% of remaining data
for train_idx, val_idx in sss_val.split(train_val_df, train_val_df['target']):
    train_df = train_val_df.iloc[train_idx]
    val_df = train_val_df.iloc[val_idx]
 

def tokenize_and_format(examples):
    # Tokenize the texts
    tokenized_inputs = tokenizer(examples['text'], padding="max_length", truncation=True, max_length=max_length)
    tokenized_inputs['label'] = list(map(int, examples['target']))
    return tokenized_inputs

# Convert pandas DataFrame to Hugging Face's Dataset
train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(val_df) 
test_dataset = Dataset.from_pandas(test_df)

# Map the tokenization function across the datasets
train_dataset = train_dataset.map(tokenize_and_format, batched=True,batch_size=16)
eval_dataset = eval_dataset.map(tokenize_and_format, batched=True,batch_size=16) 
test_dataset = test_dataset.map(tokenize_and_format, batched=True,batch_size=16)




Map: 100%|██████████| 7019/7019 [00:02<00:00, 2524.41 examples/s]


In [49]:


# %%
from sklearn.metrics import f1_score, precision_score, recall_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average='weighted')
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')
    return {
        'eval_f1': f1,
        'eval_precision': precision,
        'eval_recall': recall,
    }

 


# %%

# %%
 

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",  # Use 'epoch' to evaluate at the end of each epoch
    save_strategy="epoch",  # Also use 'epoch' to save at the end of each epoch
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,  # Load the best model at the end of training based on metric
    metric_for_best_model='f1',  # Define the metric for evaluating the best model
    logging_dir='./logs',
    logging_steps=10,
    report_to=[] 
)


 

trainer = Trainer(
    model=model,
    args=training_args ,  # Here you will need to make sure that the Trainer is set up correctly
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer
)

# Start training
trainer.train()

 
 



/home/bhairavi/om/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1,Precision,Recall
1,0.855800,1.081792,0.643025,0.636911,0.668750
2,1.026000,0.938876,0.725880,0.733638,0.731875
3,0.575200,0.934383,0.739221,0.732590,0.749375


/home/bhairavi/om/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/bhairavi/om/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


TrainOutput(global_step=3399, training_loss=1.059129979898734, metrics={'train_runtime': 3733.5951, 'train_samples_per_second': 7.283, 'train_steps_per_second': 0.91, 'total_flos': 7155800400494592.0, 'train_loss': 1.059129979898734, 'epoch': 3.0})

In [50]:
saveDIR

'/home/bhairavi/om/om5/20NG/debertaV3_20NG'

In [51]:
save_directory = saveDIR
 

# Save the model
model.save_pretrained(save_directory)

# Save the tokenizer (optional, but recommended)
tokenizer.save_pretrained(save_directory)

('/home/bhairavi/om/om5/20NG/debertaV3_20NG/tokenizer_config.json',
 '/home/bhairavi/om/om5/20NG/debertaV3_20NG/special_tokens_map.json',
 '/home/bhairavi/om/om5/20NG/debertaV3_20NG/spm.model',
 '/home/bhairavi/om/om5/20NG/debertaV3_20NG/added_tokens.json',
 '/home/bhairavi/om/om5/20NG/debertaV3_20NG/tokenizer.json')

In [52]:

# %%
from colorama import Fore, Style


# %% [markdown]
# eval dataset performance so that keywords_classes can be fixed

# %%
results = trainer.evaluate()

# Predict using the trained model to get labels and predictions
predictions, labels, _ = trainer.predict(eval_dataset)
predictions = np.argmax(predictions, axis=1)


# %%
from sklearn.metrics import classification_report
# Generate the classification report
report = classification_report(
    labels,
    predictions,
    target_names=df['label'].unique() , # Adjust this line as per your dataset
    digits=4
)
print(Fore.CYAN,"keywords class evaluation detection RESULTS")
print(report)


# %% [markdown]
# skyline

# %%


/home/bhairavi/om/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/bhairavi/om/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


 keywords class evaluation detection RESULTS
                          precision    recall  f1-score   support

               rec.autos     0.5238    0.6567    0.5828        67
   comp.sys.mac.hardware     0.6456    0.6145    0.6296        83
           comp.graphics     0.6706    0.6867    0.6786        83
               sci.space     0.5644    0.6786    0.6162        84
      talk.politics.guns     0.7143    0.6790    0.6962        81
                 sci.med     0.7895    0.8721    0.8287        86
comp.sys.ibm.pc.hardware     0.8452    0.8353    0.8402        85
 comp.os.ms-windows.misc     0.8077    0.7778    0.7925        81
         rec.motorcycles     0.7159    0.7590    0.7368        83
      talk.religion.misc     0.9583    0.8415    0.8961        82
            misc.forsale     0.9080    0.9405    0.9240        84
             alt.atheism     0.8451    0.7059    0.7692        85
         sci.electronics     0.7229    0.7143    0.7186        84
          comp.windows.x     0

/home/bhairavi/om/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/bhairavi/om/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/bhairavi/om/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [53]:


# %%
print(Fore.RED +"TEST DATA IS OUR SKYLINE RESULT")
 
results = trainer.evaluate()

# Predict using the trained model to get labels and predictions
predictions, labels, _ = trainer.predict(test_dataset)
predictions = np.argmax(predictions, axis=1)


# %%
from sklearn.metrics import classification_report
# Generate the classification report
report = classification_report(
    labels,
    predictions,
    target_names=df['label'].unique() , # Adjust this line as per your dataset
    digits=4
)

print(report)


TEST DATA IS OUR SKYLINE RESULT


/home/bhairavi/om/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


                          precision    recall  f1-score   support

               rec.autos     0.4045    0.5621    0.4704       290
   comp.sys.mac.hardware     0.6927    0.6703    0.6813       370
           comp.graphics     0.6555    0.6911    0.6728       369
               sci.space     0.5825    0.6604    0.6190       374
      talk.politics.guns     0.7035    0.6177    0.6578       361
                 sci.med     0.8090    0.8243    0.8166       370
comp.sys.ibm.pc.hardware     0.8472    0.8886    0.8674       368
 comp.os.ms-windows.misc     0.8348    0.7853    0.8093       354
         rec.motorcycles     0.8000    0.7424    0.7701       361
      talk.religion.misc     0.9215    0.8661    0.8930       366
            misc.forsale     0.9119    0.9312    0.9215       378
             alt.atheism     0.8123    0.7333    0.7708       360
         sci.electronics     0.6446    0.6621    0.6532       367
          comp.windows.x     0.8861    0.8598    0.8728       371
        r